# Reconciliation Monthly Stats

We need to load in these libraries into our notebook in order to query, load, manipulate and view the data

In [ ]:
import os
from datetime import datetime, timedelta
from config import Config

%load_ext sql
%config SqlMagic.displaylimit = 5

# Parameters cell for external parameters via papermill (job running this notebook will insert a parameter cell below this). This cell has a tag of with the name "parameters" that is used by papermill

e.g.
param1 = "some_value"

This will create the connection to the database and prep the jupyter magic for SQL

In [ ]:
%sql $Config.SQLALCHEMY_DATABASE_URI

Simplest query to run to ensure our libraries are loaded and our DB connection is working

In [ ]:
%%sql
select now() AT TIME ZONE 'PST' as current_date

Query ...

In [ ]:
%%sql monthly_reconciliation_summary  <<
SELECT count(*)                   AS transaction_count,
sum(pli.total)                    AS subtotal,
sum(pli.service_fees)             AS service_fees,
sum(pli.total + pli.service_fees) AS total,
(i.payment_date)::date,
pli.description,
i.payment_method_code,
i.corp_type_code
FROM invoices i
JOIN payment_line_items pli ON i.id = pli.invoice_id
WHERE i.corp_type_code = :partner_code
AND i.invoice_status_code = 'PAID'
AND i.payment_method_code IN ('PAD', 'EJV', 'DIRECT_PAY', 'DRAWDOWN')
AND date(i.payment_date) > date(current_date - 1 - interval '1 months')
AND date(i.payment_date) <= date(current_date - 1)
GROUP BY (i.payment_date)::date, i.payment_method_code, i.corp_type_code, pli.description
ORDER BY (i.payment_date)::date, pli.description, i.payment_method_code;

Save to CSV

In [ ]:
filename_summary = os.path.join(os.getcwd(), r'data/')+partner_code+'_monthly_reconciliation_summary_' + datetime.strftime(datetime.now()-timedelta(1), '%Y-%m') +'.csv'
df_summary = monthly_reconciliation_summary.DataFrame()
with open(filename_summary, 'w') as f:
    f.write('Monthly Reconciliation Summary:\n\n')
    if df_summary.empty:
        f.write('No Data Retrieved')
    else:
        df_summary.to_csv(f, sep=',', encoding='utf-8', index=False)

In [ ]:
%%sql monthly_reconciliation_disbursed  <<
SELECT count(*)                   AS transaction_count,
sum(pli.total)                    AS sub_total,
sum(pli.service_fees)             AS service_fees,
sum(pli.total + pli.service_fees) AS total,
(i.disbursement_date)::date,
pli.description,
i.payment_method_code,
i.corp_type_code
FROM invoices i
JOIN payment_line_items pli ON i.id = pli.invoice_id
WHERE i.corp_type_code = 'VS'
AND i.invoice_status_code = 'PAID'
AND i.payment_method_code IN ('PAD', 'EJV', 'DIRECT_PAY')
AND i.disbursement_status_code = 'COMPLETED'
AND date(disbursement_date) > date(current_date - 1 - interval '1 months')
AND date(disbursement_date) <= date(current_date - 1)
GROUP BY (disbursement_date)::date, payment_method_code, corp_type_code, pli.description
ORDER BY (disbursement_date)::date, pli.description, i.payment_method_code;

Save to another CSV

In [ ]:
filename_disbursed = os.path.join(os.getcwd(), r'data/')+partner_code+'_monthly_reconciliation_disbursed_' + datetime.strftime(datetime.now()-timedelta(1), '%Y-%m') +'.csv'
df_disbursed = monthly_reconciliation_disbursed.DataFrame()
with open(filename_disbursed, 'a') as f:
    f.write('Monthly Reconciliation Disbursed:\n\n')
    if df_disbursed.empty:
        f.write('No Data Retrieved')
    else:
        df_disbursed.to_csv(f, sep=',', encoding='utf-8', index=False)
